# Amazon ElastiCache

<img src="../_assets/aws_service_icons/elasticache.svg" width="80" alt="Amazon ElastiCache">

## Goals
- Explain what ElastiCache is
- Explain what it is practically used for
- Show a simple pseudo-code example using the AWS SDK

## What ElastiCache is
**Amazon ElastiCache** is AWS’s fully managed **in-memory data store** service. It provides managed deployments of:
- **Redis** (including replication groups, clustering, persistence options, and high availability features)
- **Memcached** (simple, multi-node, in-memory caching)

AWS manages provisioning, patching, monitoring, scaling operations, and integrates with VPC networking/security (subnets, security groups).

## What it is practically used for
ElastiCache is used when you want **very low-latency reads/writes** (often sub-millisecond) for **non-primary** data that is:
- **Derived** from another source (e.g., database query results)
- **Temporary** (e.g., sessions)
- **Fast-moving** (e.g., counters, leaderboards)

Common practical use cases and patterns:
- **Cache-aside** (a.k.a. lazy loading): read from cache first; on miss, read from DB and populate cache with a TTL
- **Session store** for web apps (especially for horizontally scaled stateless app servers)
- **Rate limiting / throttling** (atomic counters)
- **Leaderboards & real-time metrics** (sorted sets, counters)
- **Pub/Sub and lightweight queues** (Redis features)
- **ML/serving**: cache features, embeddings, and inference responses to reduce latency and backend load

## Using ElastiCache with the AWS SDK (pseudo-code)
The **AWS SDK** is typically used to **provision and manage** ElastiCache resources and to **discover endpoints**. To actually read/write cached values you connect to the Redis/Memcached endpoint using a **protocol client** (Redis or Memcached library).

```python
# PSEUDO-CODE (do not run)

import boto3
# from redis import Redis  # example protocol client library

# 1) Use the AWS SDK to discover your cache endpoint (example: Redis replication group)
elasticache = boto3.client("elasticache", region_name="us-east-1")

resp = elasticache.describe_replication_groups(
    ReplicationGroupId="my-redis-rg",
)

primary = resp["ReplicationGroups"][0]["NodeGroups"][0]["PrimaryEndpoint"]
redis_host = primary["Address"]
redis_port = primary["Port"]

# 2) Connect to Redis (must have network access to the VPC/subnets/security group)
cache = Redis(host=redis_host, port=redis_port, ssl=True, password="...")

# 3) Use a common pattern: cache-aside
key = f"user_profile:{user_id}"

value = cache.get(key)
if value is None:
    value = database.query_user_profile(user_id)
    cache.set(key, value, ex=300)  # TTL (seconds)

return value
```

### Notes
- ElastiCache is commonly deployed **inside a VPC**; access is controlled by **subnets + security groups**.
- Use **TTLs** and an invalidation strategy that matches your freshness requirements.
- For higher availability with Redis, use **replication groups** with **Multi-AZ** and automatic failover.
